In [1]:
import os
import json
from copy import copy
from collections import defaultdict

from utils.initialize_font_data import all_font_names as labeled_font_names, all_json

exclusive_attributes:  []


In [2]:
all_font_names = [os.path.splitext(basename)[0]
                  for basename in os.listdir('../gwfonts')]
unlabeled_font_names = list(set(all_font_names) - set(labeled_font_names))
labeled_font_names = sorted(labeled_font_names)
unlabeled_font_names = sorted(unlabeled_font_names)
assert len(labeled_font_names) + \
    len(unlabeled_font_names) == len(all_font_names)
print('labaled fonts num: ', len(labeled_font_names))
print('unlabaled fonts num: ', len(all_font_names) - len(labeled_font_names))
print('all fonts num: ', len(all_font_names))

labaled fonts num:  200
unlabaled fonts num:  969
all fonts num:  1169


# Compare two fonts attribtues

In [3]:
for font1, font2 in zip(
    all_json["AveriaSansLibre-Italic"].items(),
    all_json["AveriaSerifLibre-Italic"].items(),
):
    print(f"{font1[0]}: {font1[1]}, {font2[1]}")

angular: 52.05, 63.65
artistic: 35.44, 53.7
attention-grabbing: 43.08, 46.23
attractive: 59.94, 78.17
bad: 21.16, 12.43
boring: 29.89, 22.43
calm: 86.25, 86.38
capitals: 0, 0
charming: 68, 88.21
clumsy: 22.8, 25.08
complex: 27.33, 32.15
cursive: 4.35, 4.51
delicate: 50.17, 59.06
disorderly: 26.25, 27.78
display: 64.85, 72.58
dramatic: 44.57, 56.21
formal: 70.85, 74.91
fresh: 90.16, 96.76
friendly: 86.26, 90.08
gentle: 93.2, 99.27
graceful: 81.12, 82.7
happy: 55.82, 63.75
italic: 91.55, 91.16
legible: 74.33, 89.36
modern: 42.07, 72.12
monospace: 0, 0
playful: 51.55, 53.89
pretentious: 38.26, 37.52
serif: 9.73, 84.6
sharp: 75.47, 77.29
sloppy: 49.83, 52.73
soft: 85.88, 86.15
strong: 50.94, 50.43
technical: 56.87, 60.73
thin: 61.28, 55.27
warm: 67.11, 78.5
wide: 43.96, 44.61


# Define font families in labaled fonts

In [4]:
font_variables = [
    'Bold',
    'Black',
    'Italic',
    'Medium',
    'Regular',
    'Semi',
    'Thin',
    'Extra',
]
def extract_font_family(font_name, font_variables=font_variables):
    for variable in font_variables:
        font_name = font_name.replace(variable, '')
    family_name = font_name.split('-')[0]
    return family_name
    
def search_fonts_with_font_family(family_name, font_names):
    result = []
    for font_name in font_names:
        tmp_font_family = extract_font_family(font_name)
        if tmp_font_family == family_name:
            result.append(font_name)
    return result

In [5]:
family_name = "Alegreya"
search_fonts_with_font_family(family_name, unlabeled_font_names)

['Alegreya-Black',
 'Alegreya-BlackItalic',
 'Alegreya-Bold',
 'Alegreya-Italic',
 'Alegreya-Regular']

In [7]:
labeled_family_names = []
labeled_family_names_count = defaultdict(int)

for font_name in labeled_font_names:
    family_name = extract_font_family(font_name)
    labeled_family_names_count[family_name] += 1
    if family_name not in labeled_family_names:
        labeled_family_names.append(family_name)
print(len(labeled_family_names))

NameError: name 'defaultdict' is not defined

In [16]:
def predict_font_attributes_from_name(font_name):
  result = {
    'italic': 0,
    'serif': 0,
    'thin': 0,
  }
  # italic
  if 'Italic' in font_name:
    result['italic'] = 1
  elif 'Oblique' in font_name:
    result['italic'] = 1
  # serif
  if 'Sans' in font_name:
    result['serif'] = 0
  elif 'Serif' in font_name:
    result['serif'] = 1
  # thin
  if 'ExtraLight' in font_name:
    result['thin'] = 0
  elif 'Light' in font_name:
    result['thin'] = 1
  elif 'Thin' in font_name:
    result['thin'] = 1
  elif 'Regular' in font_name:
    result['thin'] = 2
  elif 'Medium' in font_name:
    result['thin'] = 2
  elif 'SemiBold' in font_name:
    result['thin'] = 3
  elif 'Bold' in font_name:
    result['thin'] = 4
  elif 'ExtraBold' in font_name:
    result['thin'] = 5
  elif 'Black' in font_name:
    result['thin'] = 5
  else:
    result['thin'] = 2
  return result

def predict_actual_font_attributes(target_font_attributes, base_font_attributes, base_font_actual_attributes):
  target_font_actual_attributes = copy(base_font_actual_attributes)
  # italic
  if target_font_attributes['italic'] == base_font_attributes['italic']:
    target_font_actual_attributes['italic'] = base_font_actual_attributes['italic']
  else:
    if float(base_font_actual_attributes['italic']) >= 50.0:
      target_font_actual_attributes['italic'] = '0'
    else:
      target_font_actual_attributes['italic'] = '100'
  # serif
  if target_font_attributes['serif'] == base_font_attributes['serif']:
    target_font_actual_attributes['serif'] = base_font_actual_attributes['serif']
  else:
    if float(base_font_actual_attributes['serif']) >= 50.0:
      target_font_actual_attributes['serif'] = '0'
    else:
      target_font_actual_attributes['serif'] = '100'
  # thin
  distance = target_font_attributes['thin'] - base_font_attributes['thin']
  target_font_actual_attributes['thin'] = str(max(min(float(base_font_actual_attributes['thin']) - distance * 20, 90), 10))
  return target_font_actual_attributes

In [17]:
font1_name = "Alegreya-Regular"
font2_name = "Alegreya-BoldItalic"

font1_attributes = predict_font_attributes_from_name(font1_name)
font2_attributes = predict_font_attributes_from_name(font2_name)
print(font1_attributes)
print(font2_attributes)

# font1_actual_attributes = all_json[font1_name]
font2_actual_attributes = all_json[font2_name]

predicted_font1_actual_attributes = predict_actual_font_attributes(font1_attributes, font2_attributes, font2_actual_attributes)
print(predicted_font1_actual_attributes)
print(font2_actual_attributes)

{'italic': 0, 'serif': 0, 'thin': 2}
{'italic': 1, 'serif': 0, 'thin': 4}
{'angular': '79.5', 'artistic': '70.87', 'attention-grabbing': '70.27', 'attractive': '100', 'bad': '20.39', 'boring': '11.17', 'calm': '71.96', 'capitals': '0', 'charming': '100', 'clumsy': '27.3', 'complex': '65.96', 'cursive': '4.52', 'delicate': '42.39', 'disorderly': '30.34', 'display': '8.93', 'dramatic': '68.69', 'formal': '72.02', 'fresh': '91.28', 'friendly': '84.06', 'gentle': '86.68', 'graceful': '90.62', 'happy': '81.67', 'italic': '0', 'legible': '88.96', 'modern': '75.63', 'monospace': '0', 'playful': '62.65', 'pretentious': '63.84', 'serif': '85.6', 'sharp': '79.38', 'sloppy': '59.15', 'soft': '72.29', 'strong': '63.21', 'technical': '61.87', 'thin': '75.47999999999999', 'warm': '87.06', 'wide': '41.08'}
{'angular': '79.5', 'artistic': '70.87', 'attention-grabbing': '70.27', 'attractive': '100', 'bad': '20.39', 'boring': '11.17', 'calm': '71.96', 'capitals': '0', 'charming': '100', 'clumsy': '27.3'

In [18]:
font1_name = "Arvo-Bold"
font2_name = "Arvo-Italic"

font1_attributes = predict_font_attributes_from_name(font1_name)
font2_attributes = predict_font_attributes_from_name(font2_name)
print(font1_attributes)
print(font2_attributes)

font1_actual_attributes = all_json[font1_name]
font2_actual_attributes = all_json[font2_name]

predicted_font1_actual_attributes = predict_actual_font_attributes(font1_attributes, font2_attributes, font2_actual_attributes)
print(predicted_font1_actual_attributes)
print(font1_actual_attributes)
print(font2_actual_attributes)

{'italic': 0, 'serif': 0, 'thin': 4}
{'italic': 1, 'serif': 0, 'thin': 2}
{'angular': '86.86', 'artistic': '51.59', 'attention-grabbing': '55.03', 'attractive': '75.69', 'bad': '31.07', 'boring': '23.64', 'calm': '74.82', 'capitals': '0', 'charming': '73.24', 'clumsy': '30.18', 'complex': '37.27', 'cursive': '4.23', 'delicate': '43.82', 'disorderly': '23.75', 'display': '6.05', 'dramatic': '42.38', 'formal': '70.44', 'fresh': '93.53', 'friendly': '75.94', 'gentle': '89.13', 'graceful': '71.92', 'happy': '56.11', 'italic': '0', 'legible': '89.19', 'modern': '54.54', 'monospace': '0', 'playful': '50.53', 'pretentious': '42.95', 'serif': '81.78', 'sharp': '92.56', 'sloppy': '50.03', 'soft': '68.01', 'strong': '60.36', 'technical': '69.44', 'thin': '10', 'warm': '73.65', 'wide': '59.73'}
{'angular': '27.86', 'artistic': '37.03', 'attention-grabbing': '61.33', 'attractive': '70.08', 'bad': '17.66', 'boring': '64.93', 'calm': '57.64', 'capitals': '0', 'charming': '63.24', 'clumsy': '33.13', 

In [19]:
count = 0
predicted_unlabeled_attributes = {}
for unlabaled_font_name in unlabeled_font_names:
    unlabeled_font_attributes = predict_font_attributes_from_name(unlabaled_font_name)
    unlabeled_font_actual_attributes = None
    font_family = extract_font_family(unlabaled_font_name)
    if font_family in labeled_family_names:
        count += 1
        labeled_font_names_with_the_font_family = search_fonts_with_font_family(font_family, labeled_font_names)
        if len(labeled_font_names_with_the_font_family) >= 1:
            predicted_actual_attributes_list = []
            for labeled_font_name in labeled_font_names_with_the_font_family:
                labeled_font_attributes = predict_font_attributes_from_name(labeled_font_name)
                labeled_font_actual_attributes = all_json[labeled_font_name]
                predicted_actual_attributes = predict_actual_font_attributes(unlabeled_font_attributes, labeled_font_attributes, labeled_font_actual_attributes)
                predicted_actual_attributes_list.append(predicted_actual_attributes)
            result_predicted_actual_attributes = {
                k: sum([float(d[k]) for d in predicted_actual_attributes_list]) / len(predicted_actual_attributes_list)
                for k in predicted_actual_attributes_list[0].keys()
            }
            predicted_unlabeled_attributes[unlabaled_font_name] = result_predicted_actual_attributes
print(count)

272


In [20]:
predicted_unlabeled_attributes['Cantarell-Bold']

{'angular': 83.17,
 'artistic': 46.34,
 'attention-grabbing': 72.1,
 'attractive': 70.27,
 'bad': 36.02,
 'boring': 28.48,
 'calm': 71.63,
 'capitals': 0.0,
 'charming': 70.91,
 'clumsy': 26.27,
 'complex': 48.77,
 'cursive': 6.6,
 'delicate': 45.55,
 'disorderly': 42.69,
 'display': 8.11,
 'dramatic': 64.32,
 'formal': 53.71,
 'fresh': 86.16,
 'friendly': 75.81,
 'gentle': 85.78,
 'graceful': 70.4,
 'happy': 68.84,
 'italic': 0.0,
 'legible': 82.54,
 'modern': 61.12,
 'monospace': 0.0,
 'playful': 46.55,
 'pretentious': 55.08,
 'serif': 9.94,
 'sharp': 72.65,
 'sloppy': 70.69,
 'soft': 66.64,
 'strong': 66.25,
 'technical': 53.23,
 'thin': 36.05,
 'warm': 76.17,
 'wide': 58.56}

# Dump to Json

In [23]:
with open('../attributeData/predicted_unlabeled_attributes.json', 'w') as f:
    json.dump(predicted_unlabeled_attributes, f)